## QFAMES with transverse field Ising model.

In [ ]:
import numpy as np
from scipy import linalg as LA
from scipy.signal import find_peaks
from scipy.special import zeta

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsfonts, amssymb, amsmath, physics}')
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['font.size'] = 12



def plot_spectrum(gsEs, t, M_list, N_sample, L, Delta, h):
    # Compute G matrix around g.s.
    # Plot squared Frobenius norms of G matrix
    μ_vals = np.linspace(gsEs[0] - 0.3, gsEs[0] + 2, N_sample)
    norm2 = np.zeros_like(μ_vals, dtype=float)

    for idx, μ in enumerate(μ_vals):
        G = np.einsum('ijkl,ij->kl',M_list, np.exp(-np.subtract.outer(t, t) * 1j * μ))
        G /= N_sample ** 2
        norm2[idx] = np.linalg.norm(G, ord='fro')**2

    # Find peaks
    peaks, _ = find_peaks(norm2, height=0.5)
    
    # Plot
    plt.figure(figsize=(4.5,3.5))
    plt.plot(μ_vals, norm2, label=r'$\lVert G(\theta)\rVert_F^2$', lw=2)

    # Add vertical dashed lines indicating eigenvalues
    for idx, energy in enumerate(gsEs):
        plt.axvline(energy, color='gray', linestyle='--', linewidth=1,
                    label=r'$E_{%d} = %.2f$' % (idx + 1, energy))

    plt.xlabel(r'$\theta$', fontsize=15)
    plt.title(r'$\Delta = %.2f, h = %.2f$' % (Delta, h), fontsize=15)
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

    # loc = 'upper right'
    # bbox_to_anchor = None if Delta < 1.4 else (0.54, 1.02)
    # plt.legend(frameon=False, loc=loc, bbox_to_anchor=bbox_to_anchor, fontsize=10, handlelength=1)
    plt.legend(frameon=True, fontsize=13, handlelength=1, loc='upper right')
    plt.scatter(μ_vals[peaks[0]], norm2[peaks[0]], marker='*', color = 'red', s=50, zorder=100)
    plt.savefig('figs/XXZ_spec_L%d_Delta%.2f_h%.2f.pdf' % (L, Delta, h), bbox_inches='tight')
    plt.close()

    # Return peaks of squared Frobenius norms
    return μ_vals[peaks]


def compute_and_plot_eigs(mu, t, M_list, N_sample, L, Delta, h, title='', thr=0.2):
    # Compute multiplicity at the peak of squared Frobenius norms (mu)
    # Threshold preset to be 0.05 -- should change w.r.t. noise
    # Plot the singular values of G matrix

    G = np.einsum('ijkl,ij->kl', M_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    G /= (N_sample) ** 2

    plt.figure(figsize=(4, 3.5))
    eigvals = LA.eigvalsh(G)[::-1]


    x = np.arange(len(eigvals))
    plt.axhline(thr, color='red', linestyle='--', linewidth=2, label=r'$\tau$')
    plt.bar(x+1, eigvals, label=r'$\sigma(G(\theta^\star_1))$', color='blue', width=0.4)
    plt.xticks(x+1, fontsize=15)
    plt.yticks(fontsize=15)
    if Delta < -0.5 and h > 0.5:
        plt.legend(fontsize=15)
    plt.savefig('figs/XXZ_eigs_L%d_Delta%.2f_h%.2f.pdf' % (L, Delta, h),bbox_inches='tight')
    plt.close()

    # return multiplicity 
    return np.sum(np.where(eigvals > thr, 1, 0))


def compute_obs(mu, t, M_list, M_O_list, N_sample, multi):
    # Compute observable expectations at corresponding energy eigenstates    
    G = np.einsum('ijkl,ij->kl', M_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    G /= (N_sample) ** 2

    GO = np.einsum('ijkl,ij->kl', M_O_list, np.exp(-np.subtract.outer(t, t) * 1j * mu))
    GO /= (N_sample) ** 2


    u, s, vh = LA.svd(G)

    G = (u[:,:multi]).conj().T @ G @ (vh.conj().T)[:,:multi]
    GO = (u[:,:multi]).conj().T @ GO @ (vh.conj().T)[:,:multi]

    eigvals = LA.eigvalsh(GO, b=G)  
    
    return eigvals * 4


def plotData(fname, Delta, h, L, chi, N_sample, m, obs_names, noise_level = 0):
    t = np.loadtxt(f'{fname}_ts')
    gsEs = np.loadtxt(f'{fname}_gsEs')
    M_list = np.loadtxt(f'{fname}_M_list', dtype=np.complex128)
    M_list = M_list.reshape((N_sample, N_sample, m, m))

    
    tmax = 10
    ind_t = np.argwhere(np.abs(t) < tmax)[:, 0]
    t = t[ind_t]
    M_list = M_list[ind_t, :, :, :]
    M_list = M_list[:, ind_t, :, :]

    M_list += noise_level * np.random.randn(len(ind_t), len(ind_t), m, m)


    M_O_list = []
    for obs_idx, obs_name in enumerate(obs_names):
        l = np.loadtxt(f'{fname}_M_{obs_name}_list', dtype=np.complex128)
        l = l.reshape((N_sample, N_sample, m, m))
        l = l[ind_t, :, :, :]
        l = l[:, ind_t, :, :]
        l += noise_level * np.random.randn(len(ind_t), len(ind_t), m, m)
        M_O_list.append(l)

    peaks = plot_spectrum(gsEs, t, M_list, len(ind_t), L, Delta, h)

    μ = peaks[0]
    multi = compute_and_plot_eigs(μ, t, M_list, len(ind_t), L, Delta, h)
    print('Delta =', Delta, 'h =', h, 'multi =', multi)
    
    vals = 0

    obs_val = []
    for obs_idx, obs_name in enumerate(obs_names):
        if (multi > 0):
            vals = compute_obs(μ, t, M_list, M_O_list[obs_idx], len(ind_t), multi)
        else:
            print('Error! No eigvals detected')
        obs_val.append(vals)

    return peaks[0], multi, obs_val

In [18]:
# System size

Ls = [10]

# Parameter for XXZ: Delta, h: H = sum (SxSx + SySy + Delta SzSz) + h sum Sz
# and number of QFAMES initial states: m
# gms = [[Delta, h, 5] for Delta in [-1., -0.5, 0., 0.5, 1.0, 1.5] for h in [0., 0.5, 1.0]]
gms = [[-1., 0., 5], [0., 0., 5], [1., 0., 5], [-1., 1., 5], [2., 0., 5]]
print(gms)
# Observable names
obs_names_plot = [r'$\sigma^z_i \sigma^{z}_{i+%d}$' % r for r in range(1, 5)]
obs_names = ['SzCorr%d' % r for r in range(1, 5)]

# N_sample = 100
N_sample = 100
epsilon = 0.1
# chi = 100
chi = 100

colors = ['C1', 'C2', 'C3', 'C4', 'C5']
L = 20
print('L =', L)
peaks = []
multis = []
obs_vals = []
for Delta, h, m in gms:
    fname = f'data/xxz_chain_Delta{Delta}_h{h}_L{L}_chi{chi}_N{N_sample}'
    peak, multi, obs_val = plotData(fname, Delta, h, L, chi, N_sample, m, obs_names)
    peaks.append(peak)
    multis.append(multi)
    obs_vals.append(obs_val)

# plot Sz
# plt.figure(figsize=(4, 3.5))
# xticks = []
# xtick_labels = []
# for i in range(len(gms)):
#     for j in range(multis[i]):
#         plt.scatter(i, obs_vals[i][0][j], edgecolor = 'C0', facecolor='C0')
#     plt.vlines(i, np.min(obs_vals[i][0]), np.max(obs_vals[i][0]), color = 'grey', lw = 2, zorder = -1)
#     xtick_labels.append('(%s,%s)' % (str(gms[i][0]), str(gms[i][1])))
#     xticks.append(i)
# plt.axhline(0, color='grey', linestyle='--', linewidth=0.5)
# plt.xticks(xticks, xtick_labels)
# plt.ylabel(obs_names_plot[0])
# plt.savefig('figs/ob_%d_L%d.pdf' % (0, L),bbox_inches='tight')
# plt.close()

# Plot SzCorr
for i in range(len(gms)):
    plt.figure(figsize=(4, 3.5))
    Delta, h, m = gms[i]
    for obs_idx, obs_name in enumerate(obs_names):
        for j in range(multis[i]):
            plt.scatter(obs_idx + 1, obs_vals[i][obs_idx][j], edgecolor = 'C0', facecolor='C0', s=60)
        plt.vlines(obs_idx + 1, np.min(obs_vals[i][obs_idx]), np.max(obs_vals[i][obs_idx]), color = 'grey', lw = 2, zorder = -1)

    # XY model
    if np.abs(Delta - 0) < 1e-5 and np.abs(h - 0) < 1e-5:
        plt.scatter(2, 0, color = 'red', s = 60, marker = 'x')
        plt.scatter(4, 0, color = 'red', s = 60, marker = 'x')
        plt.scatter(1, - 4 / np.pi ** 2, color = 'red', s = 60, marker = 'x')
        plt.scatter(3, - 4 / np.pi ** 2 / 9, color = 'red', s = 60, marker = 'x')
    
    # Heisenberg model
    if np.abs(Delta - 1) < 1e-5 and np.abs(h - 0) < 1e-5:
        plt.scatter(1, 1/3 - 4/3 * np.log(2), color = 'red', s = 60, marker = 'x')
        plt.scatter(2, 1/3 - 16/3 * np.log(2) + 3 * zeta(3), color = 'red', s = 60, marker = 'x')
        plt.scatter(3, 1/3 - 12 * np.log(2) + 74/3 * zeta(3) - 56/3 * zeta(3) * np.log(2) - 6 * zeta(3) ** 2 - 125/6 * zeta(5) + 100/3*zeta(5)*np.log(2), color = 'red', s = 60, marker = 'x')
        plt.scatter(4, 0.0346535 * 4, color = 'red', s = 60, marker = 'x')

    # XXZ, Ising phase
    if np.abs(Delta - 2) < 1e-5 and np.abs(h - 0) < 1e-5:
        plt.axhline(0.538056, color='red', linestyle='--', linewidth=0.5)
        plt.axhline(-0.538056, color='red', linestyle='--', linewidth=0.5)

    plt.axhline(0, color='grey', linestyle='--', linewidth=0.5)
    plt.xlabel(r'$r$', fontsize=16)
    plt.ylabel(r'$\sigma^z_i \sigma^{z}_{i+r}$', fontsize=16)
    plt.savefig('figs/SzCorr_L%d_Delta%.2f_h%.2f.pdf' % (L, Delta, h),bbox_inches='tight')
    plt.close()

[[-1.0, 0.0, 5], [0.0, 0.0, 5], [1.0, 0.0, 5], [-1.0, 1.0, 5], [2.0, 0.0, 5]]
L = 20
Delta = -1.0 h = 0.0 multi = 5
Delta = 0.0 h = 0.0 multi = 5
Delta = 1.0 h = 0.0 multi = 4
Delta = -1.0 h = 1.0 multi = 1
Delta = 2.0 h = 0.0 multi = 2
